In [1]:
import os
PROJ_ROOT = os.path.abspath(os.path.join(os.pardir))
print(PROJ_ROOT)

/home/bruno/Desktop/coarse-discourse-validation


In [2]:
import pandas as pd

df_path = os.path.join(PROJ_ROOT, 'data', 'interim', 'clean_data.pkl')

df = pd.read_pickle(df_path)

df.head()

,annotations,author,body,id,in_reply_to,is_self_post,majority_link,majority_type,post_depth,subreddit,thread_title,url
0,"[{'link_to_post': 'none', 'main_type': 'announ...",Keatonus,"Alright guys, little background about myself. ...",t3_omv7p,NaN,True,none,announcement,0.0,100sets,"Male, 23 years old. Going for 100 sets!",https://www.reddit.com/r/100sets/comments/omv7...
1,"[{'link_to_post': 't3_omv7p', 'main_type': 'el...",Keatonus,**January 16th 3 Sets:** \nWent out shopping w...,t1_c3igqif,t3_omv7p,True,t3_omv7p,elaboration,1.0,100sets,"Male, 23 years old. Going for 100 sets!",NaN
3,"[{'link_to_post': 't3_omv7p', 'main_type': 'ap...",answerorreply,"dude, these sets are awesome. You're doing gre...",t1_c3ij8z0,t3_omv7p,True,t3_omv7p,appreciation,1.0,100sets,"Male, 23 years old. Going for 100 sets!",NaN
4,"[{'link_to_post': 't1_c3ij8z0', 'main_type': '...",Keatonus,"Thanks man! Yeah I'm trying to just keep the ""...",t1_c3inx9t,t1_c3ij8z0,True,t1_c3ij8z0,appreciation,2.0,100sets,"Male, 23 years old. Going for 100 sets!",NaN
5,"[{'link_to_post': 't3_omv7p', 'main_type': 'el...",Keatonus,Ok. Update! Sorry I haven't been doing this da...,t1_c3k2lc1,t3_omv7p,True,t3_omv7p,elaboration,1.0,100sets,"Male, 23 years old. Going for 100 sets!",NaN


# Features Exploration

In here, we'll be working on the features described on the paper.

## Structure

> We calculate several features related to the structure of the comment and its position. One feature is the depth of the comment according to Reddit’s threaded structure, which we collect as both a raw count and normalized by the number of comments in the discussion. We also calculate number of sentences, number of words, and number of characters of both the body and the title of the comment. We computed these values for both the current comment and the parent comment.

### Post depth

### Number of sentences, words and characters